<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# EU government bond yields - data exploration exercise

In this exercise we'll use a dataset of long term government bond yields from [data.europa.eu](https://data.europa.eu/euodp/en/data/dataset/JmCLAeHrvXI80AhjeV3zYQ) to practise exploring data with `pandas`.

### 1. Load in the data

Open the dataset `eu-govt-bonds.tsv` in the `data` folder with a text editing tool like Notepad to examine its structure. This will help determine what options to use when loading the data in. Things to consider:

- is there a header row?
- what is the separator used to separate the different items of data?

Examine the top 5 rows once you've loaded the data in

In [1]:
import pandas as pd

df = pd.read_csv("../data/eu-govt-bonds.tsv", sep="\t")
df.head()

,"int_rt,geo\time",2019M02,2019M03,2019M04,2019M05,2019M06,2019M07,2019M08,2019M09,2019M10,2019M11,2019M12,2020M01
0,"MCBY,AT",0.45,0.38,0.31,0.24,0.03,-0.10,-0.37,-0.30,-0.20,-0.09,-0.04,-0.09
1,"MCBY,BE",0.69,0.54,0.47,0.41,0.15,0.00,-0.28,-0.24,-0.16,-0.04,0.01,-0.03
2,"MCBY,BG",0.68,0.67,0.50,0.48,0.32,0.43,0.35,0.35,0.25,0.22,0.18,0.15
3,"MCBY,CY",2.00,1.74,1.49,1.34,0.82,0.66,0.44,0.48,0.51,0.58,0.57,0.61
4,"MCBY,CZ",1.76,1.82,1.82,1.86,1.58,1.36,0.99,1.24,1.32,1.47,1.51,1.62


### 2. Fix the column names

Try to select just the column for January 2020. Do you get an error? If so, why?

Print the column names to see what the problem is, and fix it.

*Hint: when you view a table in Jupyter, you won't necessarily see extra whitespace!*

In [2]:
"2020M01" in df.columns

False

Strange, it certainly looks like the column should be called "2020M01"...

In [3]:
print(df.columns)

Index(['int_rt,geo\time', '2019M02 ', '2019M03 ', '2019M04 ', '2019M05 ',
       '2019M06 ', '2019M07 ', '2019M08 ', '2019M09 ', '2019M10 ', '2019M11 ',
       '2019M12 ', '2020M01 '],
      dtype='object')


In [4]:
# you can use a list comprehension to trim whitespace for all column names
df.columns = [c.strip() for c in df.columns]
df["2020M01"].head()

0    -0.09 
1    -0.03 
2     0.15 
3     0.61 
4     1.62 
Name: 2020M01, dtype: object

### 3. Extract the country code into its own column

The first column of data has the two-letter country code at the end of each value. Extract this into a new column called "country_code", and drop the original column.

In [5]:
df["country_code"] = df["int_rt,geo\\time"].str[-2:]
df.drop("int_rt,geo\\time", axis=1, inplace=True)
df.head()

,2019M02,2019M03,2019M04,2019M05,2019M06,2019M07,2019M08,2019M09,2019M10,2019M11,2019M12,2020M01,country_code
0,0.45,0.38,0.31,0.24,0.03,-0.10,-0.37,-0.30,-0.20,-0.09,-0.04,-0.09,AT
1,0.69,0.54,0.47,0.41,0.15,0.00,-0.28,-0.24,-0.16,-0.04,0.01,-0.03,BE
2,0.68,0.67,0.50,0.48,0.32,0.43,0.35,0.35,0.25,0.22,0.18,0.15,BG
3,2.00,1.74,1.49,1.34,0.82,0.66,0.44,0.48,0.51,0.58,0.57,0.61,CY
4,1.76,1.82,1.82,1.86,1.58,1.36,0.99,1.24,1.32,1.47,1.51,1.62,CZ


### 4. Check (and fix) the data types

Print the data types of the dataset. Do you notice anything strange?

One of the countries is causing all the columns to register as non-numeric. Find the country with invalid values and remove it from the dataset.

Once you've removed it, convert the numeric columns to be the correct numeric type.

In [6]:
df.dtypes

2019M02         object
2019M03         object
2019M04         object
2019M05         object
2019M06         object
2019M07         object
2019M08         object
2019M09         object
2019M10         object
2019M11         object
2019M12         object
2020M01         object
country_code    object
dtype: object

They should be "float" not "object" - the latter suggests there are string values somewhere...

One trick when suspecting there are non-numeric values in a numeric column is to sort the column and examine the start and end.

In [7]:
df["2019M02"].sort_values().head()

6     0.05 
5     0.06 
23    0.24 
20    0.26 
19    0.31 
Name: 2019M02, dtype: object

In [8]:
df["2019M02"].sort_values().tail()

24    2.69 
18    2.81 
9     3.84 
26    4.79 
8       : z
Name: 2019M02, dtype: object

There we go, one value that's something like ": z".

We could either cheat and find the culprit in Notepad, or do it in code:

In [9]:
# extract that last value (so we don't have to guess how many spaces there are in it)
offending_value = df["2019M02"].sort_values().values[-1]
print(offending_value)

: z


In [10]:
df.loc[df["2019M02"] == offending_value, "country_code"]

8    EE
Name: country_code, dtype: object

So it's Estonia. Let's remove it from the data and convert the columns to floats

In [11]:
df.drop(df[df["country_code"] == "EE"].index, axis=0, inplace=True)

# use a list comprehension to find the column names that should be floats
numeric_cols = [c for c in df.columns if c.startswith("20")]
for c in numeric_cols:
    # convert to float
    df[c] = df[c].astype(float)
    
# verify
df.dtypes

2019M02         float64
2019M03         float64
2019M04         float64
2019M05         float64
2019M06         float64
2019M07         float64
2019M08         float64
2019M09         float64
2019M10         float64
2019M11         float64
2019M12         float64
2020M01         float64
country_code     object
dtype: object

### 5. Load in a second dataset, of country codes and their matching country names 

There is a dataset in the `data` folder called `country-codes.csv` - load this into a new DataFrame.

***Note: if you encounter an error when loading the dataset, it is due to special characters appearing in the file, namely the "Å" in Åland Islands. To bypass this, look at the `encoding` parameter when reading in your csv. [This article](https://docs.python.org/3/library/codecs.html#standard-encodings) details all the possible encodings you can use - look for something that deals with Western European languages. ***

In [12]:
df_codes = pd.read_csv("../data/country-codes.csv", encoding="latin-1")
df_codes.head()

,Country,Alpha-2 code,Alpha-3 code
0,Afghanistan,AF,AFG
1,Albania,AL,ALB
2,Algeria,DZ,DZA
3,American Samoa,AS,ASM
4,Andorra,AD,AND


### 6. Use the country code to get the country name

Merge the rate dataset with country codes to add the country name to your original DataFrame.

***Note: perhaps not all values in your `country_code` column map to the `country-codes` file, so choose your join type wisely! Hint: look at the `how` parameter in the pandas `merge` function.***

In [13]:
print(len(df))
df_merged = df.merge(df_codes, left_on=["country_code"], right_on=["Alpha-2 code"], how="left")
print(len(df_merged))
# remove some unnecessary columns
df_merged.drop(["Alpha-2 code", "Alpha-3 code"], axis=1, inplace=True)
df_merged.head()

30
30


,2019M02,2019M03,2019M04,2019M05,2019M06,2019M07,2019M08,2019M09,2019M10,2019M11,2019M12,2020M01,country_code,Country
0,0.45,0.38,0.31,0.24,0.03,-0.10,-0.37,-0.30,-0.20,-0.09,-0.04,-0.09,AT,Austria
1,0.69,0.54,0.47,0.41,0.15,0.00,-0.28,-0.24,-0.16,-0.04,0.01,-0.03,BE,Belgium
2,0.68,0.67,0.50,0.48,0.32,0.43,0.35,0.35,0.25,0.22,0.18,0.15,BG,Bulgaria
3,2.00,1.74,1.49,1.34,0.82,0.66,0.44,0.48,0.51,0.58,0.57,0.61,CY,Cyprus
4,1.76,1.82,1.82,1.86,1.58,1.36,0.99,1.24,1.32,1.47,1.51,1.62,CZ,Czechia


### 7. Check for missing data

Is there any missing data in any of the columns? Which country codes were you not able to match to a row in the reference file?

In [14]:
df_merged.isnull().sum()

2019M02         0
2019M03         0
2019M04         0
2019M05         0
2019M06         0
2019M07         0
2019M08         0
2019M09         0
2019M10         0
2019M11         0
2019M12         0
2020M01         0
country_code    0
Country         5
dtype: int64

In [15]:
df_merged[df_merged["Country"].isnull()]

,2019M02,2019M03,2019M04,2019M05,2019M06,2019M07,2019M08,2019M09,2019M10,2019M11,2019M12,2020M01,country_code,Country
7,0.95,0.84,0.78,0.70,0.44,0.23,-0.03,-0.07,0.02,0.17,0.23,0.19,EA,NaN
8,3.84,3.76,3.42,3.37,2.67,2.16,1.98,1.50,1.34,1.36,1.42,1.34,EL,NaN
10,1.18,1.11,1.05,0.99,0.71,0.51,0.25,0.23,0.31,0.45,0.51,0.49,19,NaN
11,1.19,1.11,1.07,1.00,0.73,0.54,0.29,0.28,0.35,0.49,0.54,0.51,28,NaN
29,1.20,1.14,1.15,1.06,0.84,0.73,0.49,0.58,0.61,0.73,0.78,0.67,UK,NaN


### 8. Explicitly add the country name for the UK

The country code for the UK is "GB", but in the dataset it was coded as "UK" so the join won't have worked for that record.

Explicitly add the country name for the UK for the row where the `country_code` is `UK`

In [16]:
df_merged.loc[df_merged["country_code"] == "UK", "Country"] = "United Kingdom"

### 9. Find the country with the highest and lowest rates in January 2020

In [17]:
# Option 1: sort and get head() and tail()
jan_rates = df_merged[["Country", "2020M01"]].sort_values("2020M01")
print(jan_rates.head(1))
print(jan_rates.tail(1))

   Country  2020M01
5  Germany    -0.31
    Country  2020M01
25  Romania     4.28


In [18]:
# Option 2
# another option is to make the country name the index (the unique/primary key)
# and use .idxmax to find "the index of the highest value" (and idxmin for the minimum)
# we can also combine the two by using .agg and passing a list of aggregations we want to perform together
jan_rates.set_index("Country").agg(["idxmin", "idxmax"])

,2020M01
idxmin,Germany
idxmax,Romania


### 10. Find the country with the smallest *absolute* difference in rate between the first and last months of the dataset

In [19]:
df_merged["diff"] = abs(df_merged["2020M01"] - df_merged["2019M02"])
df_merged.sort_values("diff").head(1)

,2019M02,2019M03,2019M04,2019M05,2019M06,2019M07,2019M08,2019M09,2019M10,2019M11,2019M12,2020M01,country_code,Country,diff
18,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,LT,Lithuania,0.0


In [25]:
df_merged.drop("diff", axis=1, inplace=True)

### 11. Export your joined/cleaned dataset for use in later exercises

You can use `index=False` for the output to not contain the index column

`df_merged.to_csv("path/to/csv", index=False)`

In [26]:
df_merged.to_csv("../data/eu-govt-bonds-cleaned.csv", index=False)

### 12. [Advanced] Calculate the mean and median rates per country

This is trickier than it looks because you want the mean and median rates, but rates are in different columns.

One approach is to make the country the index (primary/unique key) and calculate the mean/median of the numeric columns only (so ignoring country code for example). Calling `.mean()` on an entire dataset will calculate the mean *per column* whereas you want the mean *per row*. There is a parameter within `.mean()` that lets you change the direction of the aggregation...

Bonus: do this in the same command, outputting a table with a row per country and a column for the mean and another for the median. *Hint: there's a built in `pandas` function that lets you combine aggregations*

In [21]:
# Set the index to country AND country code (to allow for codes that didn't match to a country)
# and calculate the mean, but along axis=1 meaning per row, not per column
df_merged.set_index(["country_code", "Country"]).mean(axis=1)

country_code  Country          
AT            Austria              0.058462
BE            Belgium              0.172308
BG            Bulgaria             0.393077
CY            Cyprus               0.971538
CZ            Czechia              1.422308
DE            Germany             -0.238462
DK            Denmark             -0.178462
EA            NaN                  0.400769
EL            NaN                  2.358462
ES            Spain                0.605385
19            NaN                  0.652308
28            NaN                  0.675385
FI            Finland              0.052308
FR            France               0.112308
HR            Croatia              1.194615
HU            Hungary              2.262308
IE            Ireland              0.301538
IT            Italy                1.803077
LT            Lithuania            0.286154
LU            Luxembourg          -0.120000
LV            Latvia               0.305385
MT            Malta                0.615385


We can do the same as above but with `.median` OR we could combine the two using `.agg`

In [22]:
# agg takes a list of valid aggregations
df_merged.set_index(["country_code", "Country"]).agg(["mean", "median"], axis=1)

,,mean,median
AT,Austria,0.058462,-0.04
BE,Belgium,0.172308,0.01
BG,Bulgaria,0.393077,0.35
CY,Cyprus,0.971538,0.66
CZ,Czechia,1.422308,1.51
DE,Germany,-0.238462,-0.31
DK,Denmark,-0.178462,-0.26
EA,NaN,0.400769,0.23
EL,NaN,2.358462,2.16
ES,Spain,0.605385,0.44


### 13. [Advanced] To make calculations easier, convert the data to a *narrow* format

The idea is rather than having a column per month, have a "month" column and have all the values stored in it. The data should therefore look like this:

| Country | Month   | Rate |
|---------|---------|------|
| Austria | 2019M02 | 0.45 |
| Austria | 2019M03 | 0.38 |

Use [this pandas article](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html) to help you find the right command.

You can read more about wide vs. narrow data here: [https://en.wikipedia.org/wiki/Wide_and_narrow_data](https://en.wikipedia.org/wiki/Wide_and_narrow_data)

In [23]:
# Python trick: put brackets around your entire expression and you can then lay it out over multiple lines
df_stacked = (
    df_merged
     .set_index(["country_code", "Country"])
     .stack() # this puts all the month columns in one column
     .reset_index() # convert the resulting Series to a DataFrame
     .rename(columns={"level_2": "month", 0: "rate"}) # rename the resulting default column names
)

df_stacked.head()

,country_code,Country,month,rate
0,AT,Austria,2019M02,0.45
1,AT,Austria,2019M03,0.38
2,AT,Austria,2019M04,0.31
3,AT,Austria,2019M05,0.24
4,AT,Austria,2019M06,0.03


### 14. Calculate the mean and median rate for each country again

Hopefully you'll agree it's more natural to do it with a narrow dataset rather than a wide one - just one difference between the two formats!

In [24]:
df_stacked.groupby("Country")["rate"].agg(["mean", "median"])

,mean,median
Country,,
Austria,0.058462,-0.04
Belgium,0.172308,0.01
Bulgaria,0.393077,0.35
Croatia,1.194615,1.06
Cyprus,0.971538,0.66
Czechia,1.422308,1.51
Denmark,-0.178462,-0.26
Finland,0.052308,-0.04
France,0.112308,0.04
